In [44]:
from __future__ import print_function

import numpy as np
import pandas as pd
from keras.models import Model
from keras.utils import np_utils
import keras 
from keras.callbacks import ReduceLROnPlateau

# read data and preprocess data

In [45]:
def readucr(filename):
    data = np.loadtxt(filename, delimiter = ',')
    Y = data[:,0]
    X = data[:,1:]
    return X, Y
  
foder = u"E:/Jupyter File/UCR_TS_Archive_2015/"

#flist = ['Adiac', 'Beef', 'CBF', 'ChlorineConcentration', 'CinC_ECG_torso', 'Coffee', 'Cricket_X', 'Cricket_Y', 'Cricket_Z', 
#'DiatomSizeReduction', 'ECGFiveDays', 'FaceAll', 'FaceFour', 'FacesUCR', '50words', 'FISH', 'Gun_Point', 'Haptics', 
#'InlineSkate', 'ItalyPowerDemand', 'Lighting2', 'Lighting7', 'MALLAT', 'MedicalImages', 'MoteStrain', 'NonInvasiveFatalECG_Thorax1', 
#'NonInvasiveFatalECG_Thorax2', 'OliveOil', 'OSULeaf', 'SonyAIBORobotSurface', 'SonyAIBORobotSurfaceII', 'StarLightCurves', 'SwedishLeaf', 'Symbols', 
#'synthetic_control', 'Trace', 'TwoLeadECG', 'Two_Patterns', 'uWaveGestureLibrary_X', 'uWaveGestureLibrary_Y', 'uWaveGestureLibrary_Z', 'wafer', 'WordsSynonyms', 'yoga']

#flist  = ['MiddlePhalanxTW']
#for each in flist:
each = 'Adiac'

fname = each
x_train, y_train = readucr(foder+fname+'/'+fname+'_TRAIN')
x_test, y_test = readucr(foder+fname+'/'+fname+'_TEST')

###################################################################################################################
'''模型变量'''
nb_epochs = 40
nb_feature = 0
nb_classes = len(np.unique(y_test))
nb_feature = x_train.shape[1]
batch_size = min(x_train.shape[0]/10, 16)
####################################################################################################################

y_train = (y_train - y_train.min())/(y_train.max()-y_train.min())*(nb_classes-1)
y_test = (y_test - y_test.min())/(y_test.max()-y_test.min())*(nb_classes-1)
Y_train = np_utils.to_categorical(y_train, nb_classes)
Y_test = np_utils.to_categorical(y_test, nb_classes)

x_train_mean = x_train.mean()
x_train_std = x_train.std()
x_train = (x_train - x_train_mean)/(x_train_std) 
x_test = (x_test - x_train_mean)/(x_train_std)

x_train = x_train.reshape(x_train.shape + (1,1,))
x_test = x_test.reshape(x_test.shape + (1,1,))

In [46]:
print("# x_train.shape: ",x_train.shape)
print("# x_test.shape: ",x_test.shape)

print("# Y_train.shape: ",Y_train.shape)
print("# Y_test.shape: ",Y_test.shape)

# x_train.shape:  (390, 176, 1, 1)
# x_test.shape:  (391, 176, 1, 1)
# Y_train.shape:  (390, 37)
# Y_test.shape:  (391, 37)


# Model_Sequential模型

In [ ]:
'''CNN'''

In [24]:
import numpy as np
np.random.seed(1337)
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D
from keras.utils import np_utils
from keras import backend as K

# 全局变量
batch_size = batch_size
nb_classses = nb_classes
epochs = nb_epochs

# input image dimensions
img_rows, img_cols = x_train.shape[1], x_train.shape[2]
# number of convolutional filters to use
nb_filters = 32
# size of pooling area for max pooling
pool_size = (3,1)
# convolution kernel size
kernel_size = (8,1)
kernel_size_2 = (3,1)
input_shape = (x_train.shape[1:])
print(input_shape)

(176, 1, 1)


In [25]:
#构建模型  1
model = Sequential()  

model.add(Convolution2D(nb_filters, (kernel_size[0], kernel_size[1]), padding='same', input_shape=input_shape)) # 卷积层1  
model.add(Activation('relu')) #激活层  

model.add(Convolution2D(nb_filters, (kernel_size[0], kernel_size[1]))) #卷积层2  
model.add(Activation('relu')) #激活层

model.add(Convolution2D(128, (kernel_size_2[0], kernel_size_2[1]))) #卷积层3  
model.add(Activation('relu')) #激活层

model.add(MaxPooling2D(pool_size=pool_size)) #池化层  
model.add(Dropout(0.25)) #神经元随机失活  
model.add(Flatten()) #拉成一维数据  
model.add(Dense(128)) #全连接层1  
model.add(Activation('relu')) #激活层  
model.add(Dropout(0.5)) #随机失活  
model.add(Dense(nb_classes)) #全连接层2  
model.add(Activation('softmax')) #Softmax评分  
  
#编译模型  
model.compile(loss='categorical_crossentropy',  
              optimizer='adadelta',  
              metrics=['accuracy'])  

#训练模型  
model.fit(x_train, Y_train, batch_size=batch_size, epochs=epochs,  
          verbose=1, validation_data=(x_test, Y_test))  
#评估模型  
score = model.evaluate(x_test, Y_test, verbose=0)  
print('Test score:', score[0])  
print('Test accuracy:', score[1])  

Train on 390 samples, validate on 391 samples
Epoch 1/40
390/390 [==============================] - 1s - loss: 3.6379 - acc: 0.0154 - val_loss: 3.6149 - val_acc: 0.0179
Epoch 2/40
390/390 [==============================] - 1s - loss: 3.6022 - acc: 0.0538 - val_loss: 3.6102 - val_acc: 0.0358
Epoch 3/40
390/390 [==============================] - 1s - loss: 3.6029 - acc: 0.0487 - val_loss: 3.6088 - val_acc: 0.0409
Epoch 4/40
390/390 [==============================] - 1s - loss: 3.5801 - acc: 0.0385 - val_loss: 3.6019 - val_acc: 0.0742
Epoch 5/40
390/390 [==============================] - 1s - loss: 3.5829 - acc: 0.0462 - val_loss: 3.5862 - val_acc: 0.0716
Epoch 6/40
390/390 [==============================] - 1s - loss: 3.5657 - acc: 0.0615 - val_loss: 3.5840 - val_acc: 0.0997
Epoch 7/40
390/390 [==============================] - 1s - loss: 3.5539 - acc: 0.0769 - val_loss: 3.5517 - val_acc: 0.0742
Epoch 8/40
390/390 [==============================] - 1s - loss: 3.5228 - acc: 0.0795 - val_l

# Model_函数式模型

In [55]:
from __future__ import print_function
 
from keras.models import Model
from keras.utils import np_utils
import numpy as np
import pandas as pd
import keras 
from keras.layers import LSTM
from keras.callbacks import ReduceLROnPlateau 

filters_1 = 128
filters_2 = 256
filters_3 = 128

kernel_size_1 = (8,1)
kernel_size_2 = (5,1)
kernel_size_3 = (3,1)

x = keras.layers.Input(x_train.shape[1:])

#    drop_out = Dropout(0.2)(x)
conv1 = keras.layers.Conv2D(128, (8, 1), border_mode='same')(x)
conv1 = keras.layers.normalization.BatchNormalization()(conv1)
conv1 = keras.layers.Activation('relu')(conv1)

print("##################################")
print("输出测试 conv1 —— feature map: ")
print(conv1)
print("##################################")

#    drop_out = Dropout(0.2)(conv1)
conv2 = keras.layers.Conv2D(256, (5, 1), border_mode='same')(conv1)
conv2 = keras.layers.normalization.BatchNormalization()(conv2)
conv2 = keras.layers.Activation('relu')(conv2)

print("输出测试 conv2 —— feature map: ")
print(conv2)
print("##################################")

#    drop_out = Dropout(0.2)(conv2)
conv3 = keras.layers.Conv2D(128, (3, 1), border_mode='same')(conv2)
conv3 = keras.layers.normalization.BatchNormalization()(conv3)
conv3 = keras.layers.Activation('relu')(conv3)

print("输出测试 conv3 —— feature map: ")
print(conv3)
print("###################################")

full = keras.layers.Reshape(conv3.shape[1], conv3.shape[2], conv3.shape[3])(conv3)
full = LSTM(8, input_shape=(full.shape[1], full.shape[2]), return_sequences=True)(conv3)

#full = keras.layers.pooling.GlobalAveragePooling2D()(conv3)

out = keras.layers.Dense(nb_classes, activation='softmax')(full)

model = Model(input=x, output=out)
 
optimizer = keras.optimizers.Adam()
model.compile(loss='categorical_crossentropy',
              optimizer=optimizer,
              metrics=['accuracy'])
 
reduce_lr = ReduceLROnPlateau(monitor = 'loss', factor=0.5,
                  patience=50, min_lr=0.0001) 
hist = model.fit(x_train, Y_train, batch_size=batch_size, nb_epoch=nb_epochs,
          verbose=1, validation_data=(x_test, Y_test), callbacks = [reduce_lr])
#Print the testing results which has the lowest training loss.
log = pd.DataFrame(hist.history)
print(log.loc[log['loss'].idxmin]['loss'], log.loc[log['loss'].idxmin]['val_acc'])

C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:22: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(128, (8, 1), padding="same")`
C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:32: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(256, (5, 1), padding="same")`
C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:41: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(128, (3, 1), padding="same")`


##################################
输出测试 conv1 —— feature map: 
Tensor("activation_49/Relu:0", shape=(?, 176, 1, 128), dtype=float32)
##################################
输出测试 conv2 —— feature map: 
Tensor("activation_50/Relu:0", shape=(?, 176, 1, 256), dtype=float32)
##################################
输出测试 conv3 —— feature map: 
Tensor("activation_51/Relu:0", shape=(?, 176, 1, 128), dtype=float32)
###################################


TypeError: __init__() takes 2 positional arguments but 4 were given

In [28]:
print("x_train.shape[1:]",x_train.shape[1:])
x_train[0]

x_train.shape[1:] (176, 1, 1)


array([[[ 1.60255907]],

       [[ 1.60396307]],

       [[ 1.57498062]],

       [[ 1.55492356]],

       [[ 1.5117006 ]],

       [[ 1.43839205]],

       [[ 1.37290575]],

       [[ 1.30902402]],

       [[ 1.21375299]],

       [[ 1.11988595]],

       [[ 1.02692149]],

       [[ 0.92862184]],

       [[ 0.83047262]],

       [[ 0.74132902]],

       [[ 0.64488466]],

       [[ 0.55801754]],

       [[ 0.46427084]],

       [[ 0.37063446]],

       [[ 0.27922442]],

       [[ 0.18636023]],

       [[ 0.09580462]],

       [[ 0.01067644]],

       [[-0.0810866 ]],

       [[-0.16602223]],

       [[-0.24418459]],

       [[-0.33638689]],

       [[-0.42732559]],

       [[-0.49890924]],

       [[-0.56846712]],

       [[-0.65171394]],

       [[-0.73433899]],

       [[-0.78001895]],

       [[-0.8453849 ]],

       [[-0.91991693]],

       [[-0.99078855]],

       [[-1.05800978]],

       [[-1.09992903]],

       [[-1.15037254]],

       [[-1.20181889]],

       [[-1.24674671]],



In [31]:
# Generate dummy data
t_x_train = np.random.random((100, 10))
t_y_train = np.random.randint(2, size=(100, 1))
t_x_test = np.random.random((10, 10))
t_y_test = np.random.randint(2, size=(10, 1))

print("# t_x_train.shape: ",t_x_train.shape)
print("# t_x_test.shape: ",t_x_test.shape)

print("# t_y_train.shape: ",t_y_train.shape)
print("# t_y_test.shape: ",t_y_test.shape)

t_x_train

# t_x_train.shape:  (100, 10)
# t_x_test.shape:  (10, 10)
# t_y_train.shape:  (100, 1)
# t_y_test.shape:  (10, 1)


array([[ 0.48530653,  0.6814032 ,  0.40589211,  0.2037158 ,  0.71807214,
         0.65999421,  0.91126138,  0.40701753,  0.60189854,  0.87693833],
       [ 0.67842036,  0.26040035,  0.83243133,  0.47147041,  0.22289942,
         0.51934516,  0.52626243,  0.41556139,  0.58881512,  0.10532688],
       [ 0.19834607,  0.39457151,  0.21891345,  0.31202382,  0.40049868,
         0.5606715 ,  0.3102617 ,  0.89832663,  0.95034988,  0.12046023],
       [ 0.0836587 ,  0.17671227,  0.12188595,  0.83126026,  0.30768291,
         0.62209125,  0.5681506 ,  0.46330067,  0.29300683,  0.42418595],
       [ 0.285441  ,  0.39636956,  0.72358094,  0.45513391,  0.45833962,
         0.62009232,  0.54833128,  0.33363549,  0.59364411,  0.93730216],
       [ 0.15806044,  0.57582786,  0.91338765,  0.82912605,  0.54902452,
         0.68220497,  0.2060992 ,  0.2293142 ,  0.32726642,  0.74304278],
       [ 0.00464153,  0.08535252,  0.2613706 ,  0.03583547,  0.25484007,
         0.08784926,  0.28345024,  0.79493829

In [37]:
t2_x_train = np.array(np.random.random((176,1,1)))
t3_x_train = np.array(np.random.random((176,1,5)))

print(t2_x_train)
print(t3_x_train)

[[[ 0.63621156]]

 [[ 0.73850751]]

 [[ 0.72442556]]

 [[ 0.22950671]]

 [[ 0.26581665]]

 [[ 0.43718159]]

 [[ 0.40804502]]

 [[ 0.62224256]]

 [[ 0.8847127 ]]

 [[ 0.88246296]]

 [[ 0.76182877]]

 [[ 0.9895867 ]]

 [[ 0.19631997]]

 [[ 0.32312628]]

 [[ 0.17305013]]

 [[ 0.4389336 ]]

 [[ 0.10823884]]

 [[ 0.28657198]]

 [[ 0.78553889]]

 [[ 0.11555483]]

 [[ 0.70053189]]

 [[ 0.16701429]]

 [[ 0.365673  ]]

 [[ 0.65889432]]

 [[ 0.08175094]]

 [[ 0.76160098]]

 [[ 0.92487546]]

 [[ 0.3648803 ]]

 [[ 0.52538441]]

 [[ 0.06350893]]

 [[ 0.72104547]]

 [[ 0.14341888]]

 [[ 0.6674371 ]]

 [[ 0.80368959]]

 [[ 0.90459924]]

 [[ 0.68994845]]

 [[ 0.7091976 ]]

 [[ 0.88611276]]

 [[ 0.69100987]]

 [[ 0.95488823]]

 [[ 0.83090261]]

 [[ 0.15250495]]

 [[ 0.90302959]]

 [[ 0.82297775]]

 [[ 0.54742358]]

 [[ 0.81500486]]

 [[ 0.72165166]]

 [[ 0.99590354]]

 [[ 0.73815579]]

 [[ 0.98971747]]

 [[ 0.12836376]]

 [[ 0.29835141]]

 [[ 0.99352019]]

 [[ 0.32406991]]

 [[ 0.47585849]]

 [[ 0.6691

In [43]:
x_train.shape

(390, 176, 1, 1)

In [41]:
a = np.array([ [[1,2,3,4,5]],
               [[11,22,33,44,55]],
               [[111,222,333,444,555,666]],
               [[1111,2222,3333,4444,5555,6666]]
             ])
a.shape

(4, 1)